In [1]:
import pandas as pd
import requests
import json
import re
import itertools
import time

In [2]:
df_festival = pd.read_csv('./data/festivals.csv', keep_default_na=False)
df_festival

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...
5,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...
6,6,Derborence,N/A,Francis Reusser,Switzerland,0,cannes,1985,https://en.wikipedia.org/wiki/Derborence_(film),https://en.wikipedia.org/wiki/Francis_Reusser
7,7,Détective,Détective,Jean-Luc Godard,France,0,cannes,1985,https://en.wikipedia.org/wiki/D%C3%A9tective_(...,https://en.wikipedia.org/wiki/Jean-Luc_Godard
8,8,Farewell to the Ark,さらば箱舟,Shūji Terayama,Japan,0,cannes,1985,https://en.wikipedia.org/wiki/Farewell_to_the_Ark,https://en.wikipedia.org/wiki/Sh%C5%ABji_Terayama
9,9,Insignificance,N/A,Nicolas Roeg,United Kingdom,0,cannes,1985,https://en.wikipedia.org/wiki/Insignificance_(...,https://en.wikipedia.org/wiki/Nicolas_Roeg


In [3]:
df_festival.iloc[0:10]

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...
5,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...
6,6,Derborence,N/A,Francis Reusser,Switzerland,0,cannes,1985,https://en.wikipedia.org/wiki/Derborence_(film),https://en.wikipedia.org/wiki/Francis_Reusser
7,7,Détective,Détective,Jean-Luc Godard,France,0,cannes,1985,https://en.wikipedia.org/wiki/D%C3%A9tective_(...,https://en.wikipedia.org/wiki/Jean-Luc_Godard
8,8,Farewell to the Ark,さらば箱舟,Shūji Terayama,Japan,0,cannes,1985,https://en.wikipedia.org/wiki/Farewell_to_the_Ark,https://en.wikipedia.org/wiki/Sh%C5%ABji_Terayama
9,9,Insignificance,N/A,Nicolas Roeg,United Kingdom,0,cannes,1985,https://en.wikipedia.org/wiki/Insignificance_(...,https://en.wikipedia.org/wiki/Nicolas_Roeg


In [4]:
for link in df_festival.iloc[0:10]['link_film']:
    print(link)

https://en.wikipedia.org/wiki/Adieu_Bonaparte
https://en.wikipedia.org/wiki/Birdy_(film)
https://en.wikipedia.org/wiki/Bliss_(1985_film)
https://en.wikipedia.org/wiki/Chicken_with_Vinegar
https://en.wikipedia.org/wiki/The_Coca-Cola_Kid
https://en.wikipedia.org/wiki/Colonel_Redl
https://en.wikipedia.org/wiki/Derborence_(film)
https://en.wikipedia.org/wiki/D%C3%A9tective_(1985_film)
https://en.wikipedia.org/wiki/Farewell_to_the_Ark
https://en.wikipedia.org/wiki/Insignificance_(film)


In [5]:
def get_film_json(key, title1, title2, title3, year):
    titles = [title1, title2, title3]
    years = [year, year-1]
    
    title_year = list(itertools.product(titles, years)) # doesn't work for original titles in languages with certain characters, e.g. Egyptian Arabic 
    
    response = 'False'
    j = 0
    while response == 'False' and j < 6: # 6 because all permutations of 3 titles and 2 years
        url = 'http://www.omdbapi.com/?apikey=' + key + '&t=' + title_year[j][0] + '&y=' + str(title_year[j][1]) + '&plot=full'
        r = requests.get(url)
        film_json = r.json()
        response = film_json['Response']
        j = j + 1
    
    return film_json

In [6]:
def clean_film_json(film_json):
    dict_ratings = {'InternetMovieDatabaseRating': 'N/A',
                    'RottenTomatoesRating': 'N/A',
                    'MetacriticRating': 'N/A'}

    source = [rating['Source'].replace(' ', '') + 'Rating' for rating in film_json['Ratings']]
    value = [rating['Value'] for rating in film_json['Ratings']]

    for i in range(len(source)):
        dict_ratings[source[i]] = value[i]

    film_json.pop('Ratings')
    film_json.update(dict_ratings)
    
    return film_json

In [7]:
credentials = pd.read_csv('credentials.csv')
omdbapikey = credentials['omdbapikey'][0]
regex_paren = '_\((.*film.*)\)'

for i in df_festival['id']:
# for i in range(10):
    t0 = time.time()
    
    title_link = df_festival.iloc[i]['link_film']
    title_link = title_link.replace('https://en.wikipedia.org/wiki/', '')
    title_link = re.sub(regex_paren, '', title_link.replace('https://en.wikipedia.org/wiki/', ''))
    title_link = title_link.replace('_', '+')
    title_english = df_festival.iloc[i]['title_english'].replace(' ', '+')
    title_original = df_festival.iloc[i]['title_original'].replace(' ', '+')
    print(i)
    print(title_link)
    
    year_festival = df_festival.iloc[i]['year_festival']
    
    film_json = get_film_json(omdbapikey, title_link, title_english, title_original, year_festival)
    film_json = clean_film_json(film_json) if film_json['Response'] == 'True' else film_json
    
    with open('./data/json/' + str(i) + '.json', 'w') as file:
        json.dump(film_json, file)
    
    print(film_json)
    
    response_delay = time.time() - t0
    print(10*response_delay)
    print('\n')
    time.sleep(10*response_delay)

0
Adieu+Bonaparte
{'Title': 'Adieu Bonaparte', 'Year': '1985', 'Rated': 'N/A', 'Released': '17 May 1985', 'Runtime': '115 min', 'Genre': 'Drama, History, War', 'Director': 'Youssef Chahine', 'Writer': 'Youssef Chahine, Yusri Nasrullah', 'Actors': 'Michel Piccoli, Mohsen Mohieddin, Patrice Chéreau, Mohamad Atef', 'Plot': 'A story during the French Occupation of Egypt (1797-1803), depicting the conflict between tradintion and modernization in the context of fighting the French invadors.', 'Language': 'French, Arabic', 'Country': 'Egypt, France', 'Awards': '1 nomination.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BYWRjOTNhNDQtY2M4OS00YzM0LWI5NTctNzU1ZWI2ZmRjODRhXkEyXkFqcGdeQXVyMzI4MTk3MTY@._V1_SX300.jpg', 'Metascore': 'N/A', 'imdbRating': '6.4', 'imdbVotes': '236', 'imdbID': 'tt0088383', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True', 'InternetMovieDatabaseRating': '6.4/10', 'RottenTomatoesRating': 'N/A', 'MetacriticRa

8
Farewell+to+the+Ark
{'Title': 'Farewell to the Ark', 'Year': '1984', 'Rated': 'N/A', 'Released': '08 Sep 1984', 'Runtime': '127 min', 'Genre': 'Fantasy, Mystery, Romance', 'Director': 'Shûji Terayama', 'Writer': 'Gabriel García Márquez (novel), Rio Kishida, Shûji Terayama', 'Actors': 'Tsutomu Yamazaki, Mayumi Ogawa, Yoshio Harada, Yôko Takahashi', 'Plot': 'A surreal, isolated village sees its inhabitants gradually leave behind their mutual traditions and superstitions as they leave for the city. Among them are two cousins who love each other and who get into a quarrel with other villagers.', 'Language': 'Japanese', 'Country': 'Japan', 'Awards': '7 wins & 2 nominations.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZmFhM2Y0MzYtMjc1MS00NTEwLTkzMjUtNGU3ZTljZjQ2NGFkXkEyXkFqcGdeQXVyMzU0NzkwMDg@._V1_SX300.jpg', 'Metascore': 'N/A', 'imdbRating': '7.4', 'imdbVotes': '251', 'imdbID': 'tt0088041', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 

15
Mishima:+A+Life+in+Four+Chapters
{'Title': 'Mishima: A Life in Four Chapters', 'Year': '1985', 'Rated': 'R', 'Released': '20 Sep 1985', 'Runtime': '120 min', 'Genre': 'Biography, Drama', 'Director': 'Paul Schrader', 'Writer': 'Paul Schrader, Leonard Schrader', 'Actors': 'Ken Ogata, Masayuki Shionoya, Hiroshi Mikami, Junya Fukuda', 'Plot': 'A fictionalized account in four segments of the life of Japan\'s celebrated twentieth-century author Yukio Mishima. Three of the segments parallel events in Mishima\'s life with his novels (The Temple of the Golden Pavilion (Kinkaku-ji), Kyoko\'s House, and Runaway Horses), while the fourth depicts 25 November 1970, "The Last Day"...', 'Language': 'Japanese, English', 'Country': 'USA, Japan', 'Awards': '1 win & 1 nomination.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDM2NWU1NWEtYzBmNC00MDA4LWI2YzQtZTQ1Y2E3NjZhMDJjXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_SX300.jpg', 'Metascore': '81', 'imdbRating': '7.9', 'imdbVotes': '7,681', 'imdbID': 'tt00896

23
La+Belle+Noiseuse
{'Title': 'La Belle Noiseuse', 'Year': '1991', 'Rated': 'Unrated', 'Released': '04 Sep 1991', 'Runtime': '238 min', 'Genre': 'Drama', 'Director': 'Jacques Rivette', 'Writer': 'Pascal Bonitzer (scenario), Christine Laurent (scenario), Jacques Rivette (scenario), Honoré de Balzac (inspired by novella "Le chef-d\'oeuvre inconnu"), Pascal Bonitzer (dialogue), Christine Laurent (dialogue)', 'Actors': 'Michel Piccoli, Jane Birkin, Emmanuelle Béart, Marianne Denicourt', 'Plot': "The former famous painter Frenhofer lives quietly with his wife in his countryside residence in the French Provence. When the young artist Nicolas visits him with his girlfriend Marianne, Frenhofer decides to start working again on a painting called 'La Belle Noiseuse', which he gave up a long time ago. And he wants Marianne as a model. The ensuing creative process will change the characters' lives. It will become a struggle for truth and meaning, and the question about the limits of art will aris

29
Guilty+by+Suspicion
{'Title': 'Guilty by Suspicion', 'Year': '1991', 'Rated': 'PG-13', 'Released': '15 Mar 1991', 'Runtime': '105 min', 'Genre': 'Drama', 'Director': 'Irwin Winkler', 'Writer': 'Irwin Winkler', 'Actors': 'Robert De Niro, Annette Bening, George Wendt, Patricia Wettig', 'Plot': 'David Merrill (Robert De Niro), a fictitious 1950s Hollywood director, returns from filming abroad in France to find that his loyalty has been called into question by the House Committee on Un-American Activities and he is unable to work until cleared. Before being called, his highest priority had been his work to the extent of leaving his wife (Annette Bening) and son (Luke Edwards) alone for months at a time. He initially refuses to implicate others or himself in a private meeting with Roy Cohn and a studio lawyer. This decision initially to stick to his principles first leaves him unable to work in his profession, even with films and producers he never would have worked with before. Harassme

...

732
N/A
{'Title': 'A Estrangeira', 'Year': '1982', 'Rated': 'N/A', 'Released': '04 Mar 1983', 'Runtime': '100 min', 'Genre': 'Drama', 'Director': 'João Mário Grilo', 'Writer': 'Teresa Azevedo, Michael Graham, João Mário Grilo', 'Actors': 'Fernando Rey, Teresa Madruga, Carole Courtoy, Mariana Villar', 'Plot': 'N/A', 'Language': 'Portuguese', 'Country': 'Portugal', 'Awards': '1 nomination.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZWE2YzQ2Y2EtZmM1Mi00ZDg1LWFiMTUtNzQxODVkMWNkZjRiXkEyXkFqcGdeQXVyNDg5MjMyMTM@._V1_SX300.jpg', 'Metascore': 'N/A', 'imdbRating': '7.1', 'imdbVotes': '12', 'imdbID': 'tt0083904', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True', 'InternetMovieDatabaseRating': '7.1/10', 'RottenTomatoesRating': 'N/A', 'MetacriticRating': 'N/A'}
6.525449752807617


733
Le+Beau+Mariage
{'Title': 'A Good Marriage', 'Year': '1982', 'Rated': 'PG', 'Released': '27 Aug 1982', 'Runtime': '93 min', 'Genre': 'Comedy, Drama

741
Private+Life
{'Title': 'Chastnaya zhizn', 'Year': '1982', 'Rated': 'N/A', 'Released': '23 Aug 1982', 'Runtime': '102 min', 'Genre': 'Drama', 'Director': 'Yuli Raizman', 'Writer': 'Anatoli Grebnev, Yuli Raizman', 'Actors': 'Mikhail Ulyanov, Iya Savvina, Irina Gubanova, Tatyana Dogileva', 'Plot': 'A man gets fired from his cooshie government job. He thinks its the end of the world but slowly finds things to live for. Nominated for Oscar in the foreign language movie category.', 'Language': 'Russian', 'Country': 'Soviet Union', 'Awards': 'Nominated for 1 Oscar. Another 1 win & 1 nomination.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMWJhYjk3MjItNzg1OC00ZmJiLWE2OTMtYTBmMTAwNzNhZTMzXkEyXkFqcGdeQXVyMTc4MzI2NQ@@._V1_SX300.jpg', 'Metascore': 'N/A', 'imdbRating': '7.1', 'imdbVotes': '137', 'imdbID': 'tt0083732', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True', 'InternetMovieDatabaseRating': '7.1/10', 'RottenTomatoesRatin

749
Careful,+He+Might+Hear+You
{'Title': 'Careful, He Might Hear You', 'Year': '1983', 'Rated': 'PG', 'Released': '15 Jun 1984', 'Runtime': '110 min', 'Genre': 'Drama', 'Director': 'Carl Schultz', 'Writer': 'Sumner Locke Elliott (novel), Michael Jenkins', 'Actors': 'Wendy Hughes, Robyn Nevin, Nicholas Gledhill, John Hargreaves', 'Plot': "His mother dead, PS lives in Sydney with working-class Aunt Lila and Uncle George. When he's six or eight, his posh Aunt Vanessa descends from England. Named a co-guardian by PS's feckless, absent father, she asserts her rights and convinces Lila to have PS live with her weekdays. PS is happy at Lila's, playing with children, running about, speaking up. At Vanessa's, there's a regimen of private school, round vowels, piano and riding lessons, and lonely indoor play with fancy toys. He's miserable and when he objects, she sues for complete custody. Will anyone listen to him? And will he take on Vanessa's challenges to find out who he is and to love some

{'Response': 'False', 'Error': 'Movie not found!'}
5.60978889465332


757
N/A
{'Response': 'False', 'Error': 'Request limit reached!'}
5.1065993309021


758
Life+Is+a+Bed+of+Roses
{'Response': 'False', 'Error': 'Request limit reached!'}
13.352789878845215


759
A+Love+in+Germany
{'Response': 'False', 'Error': 'Request limit reached!'}
7.226769924163818


760
Maria+Chapdelaine
{'Response': 'False', 'Error': 'Movie not found!'}
5.304059982299805


761
The+Millennial+Bee
{'Response': 'False', 'Error': 'Request limit reached!'}
7.315559387207031


762
Mother+Mary
{'Response': 'False', 'Error': 'Request limit reached!'}
7.340662479400635


763
N/A
{'Response': 'False', 'Error': 'Request limit reached!'}
7.576198577880859


764
A+School+Outing
{'Response': 'False', 'Error': 'Request limit reached!'}
7.698700428009033


765
N/A
{'Response': 'False', 'Error': 'Request limit reached!'}
7.062492370605469


766
Streamers
{'Response': 'False', 'Error': 'Movie not found!'}
5.447759628295898


767
S

KeyboardInterrupt: 

In [ ]:
credentials = pd.read_csv('credentials.csv')
omdbapikey = credentials['omdbapikey'][0]
regex_paren = '_\((.*film.*)\)'

# for i in df_festival['id']:
for i in range(757, len(df_festival['id'])):
    t0 = time.time()
    
    title_link = df_festival.iloc[i]['link_film']
    title_link = title_link.replace('https://en.wikipedia.org/wiki/', '')
    title_link = re.sub(regex_paren, '', title_link.replace('https://en.wikipedia.org/wiki/', ''))
    title_link = title_link.replace('_', '+')
    title_english = df_festival.iloc[i]['title_english'].replace(' ', '+')
    title_original = df_festival.iloc[i]['title_original'].replace(' ', '+')
    print(i)
    print(title_link)
    
    year_festival = df_festival.iloc[i]['year_festival']
    
    film_json = get_film_json(omdbapikey, title_link, title_english, title_original, year_festival)
    film_json = clean_film_json(film_json) if film_json['Response'] == 'True' else film_json
    
    with open('./data/json/' + str(i) + '.json', 'w') as file:
        json.dump(film_json, file)
    
    print(film_json)
    
    response_delay = time.time() - t0
    print(10*response_delay)
    print('\n')
    time.sleep(10*response_delay)

In [8]:
with open('./data/json/3.json') as file:
    data = json.load(file)

In [9]:
data

{'Response': 'False', 'Error': 'Movie not found!'}

In [10]:
df = pd.DataFrame(columns = ['Title',
                             'Year',
                             'Rated',
                             'Released',
                             'Runtime',
                             'Genre',
                             'Director', 
                             'Writer',
                             'Actors',
                             'Plot',
                             'Language',
                             'Country',
                             'Awards',
                             'Poster',
                             'Metascore',
                             'imdbRating',
                             'imdbVotes',
                             'imdbID',
                             'Type',
                             'DVD',
                             'BoxOffice',
                             'Production',
                             'Website',
                             'Response',
                             'InternetMovieDatabaseRating',
                             'RottenTomatoesRating',
                             'MetacriticRating'])

In [11]:
df.append(pd.Series(data), ignore_index=True)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Type,DVD,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,Movie not found!
